## RNN Language Model

Below is a diagram of the RNN computation that we will implement below. We're plugging characters into the RNN with a 1-hot encoding and expecting it to predict the next character. In this example the training data is the string "hello", so there are 4 letters in the vocabulary: [h,e,l,o].

<img src="rnnlm.jpeg">

In [1]:
import numpy as np
np.random.seed(1337)

In [2]:
# data I/O
# get shakespeare from http://cs.stanford.edu/people/karpathy/shakespeare.txt
data = open('shakespeare.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print 'data has %d characters, %d unique.' % (data_size, vocab_size)

data has 4573338 characters, 67 unique.


In [3]:
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

In [4]:
char_to_ix['a']

41

In [5]:
# lets sample a batch of data
seq_length = 25 # number of characters in the batch
p = 220000 # point in the book to sample from
print data[p:p+seq_length] # print a chunk of data

 thing when he was young,


In [6]:
inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
print inputs
print targets

[2, 61, 49, 48, 55, 46, 2, 62, 49, 44, 55, 2, 49, 44, 2, 62, 41, 58, 2, 64, 54, 60, 55, 46, 7]
[61, 49, 48, 55, 46, 2, 62, 49, 44, 55, 2, 49, 44, 2, 62, 41, 58, 2, 64, 54, 60, 55, 46, 7, 0]


In [7]:
# lets plug the first character into the RNN
ix_input = inputs[0]
ix_target = targets[0]
# encode the input character with a 1-hot representation
x = np.zeros((vocab_size,1))
x[ix_input] = 1
print x.ravel()

[ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [8]:
# create random starting parameters
hidden_size = 10
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

In [9]:
# compute the hidden state
h_prev = np.zeros((hidden_size, 1))
h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h_prev + bh))
print h.ravel()

[-0.00321813  0.00640499  0.00424507  0.00783602 -0.00428041  0.00900686
 -0.01618631  0.00071848 -0.01342806 -0.0048051 ]


In [10]:
# compute the scores for next character
y = np.dot(Why, h) + by
print y.ravel()

[  3.83078834e-05   3.73893809e-05  -1.07305639e-04   2.67774461e-04
   1.19325035e-04  -3.37354220e-04  -3.34852820e-04   3.32538588e-04
   3.18794726e-04  -1.03554566e-05   1.36845629e-04  -5.38710213e-04
  -4.57711660e-04   2.99868693e-04   1.68690516e-04   1.78520042e-04
   2.36693532e-04  -7.03401501e-05  -7.16902773e-05   1.27203335e-04
   8.04252112e-05   1.59279072e-04   2.17314448e-04  -2.72573982e-05
  -7.02211895e-04  -2.42104646e-05   5.30702842e-04   1.94948893e-04
  -9.90711082e-05   4.03627462e-04  -2.93323333e-04   3.93383233e-04
  -4.05743771e-04  -3.38345251e-04  -5.29097616e-05  -4.80911954e-05
   1.33355225e-04   4.75679274e-04   6.58103198e-05   1.94801460e-04
  -2.20233399e-04   3.08235108e-04  -2.53968781e-05   1.44684905e-04
  -8.29930806e-05   5.83661343e-04   7.77516275e-05   2.55059906e-04
   1.81287457e-04   2.64252428e-04   1.78958318e-04  -2.92440820e-04
  -3.40954240e-04   8.32849829e-05  -6.05381172e-05   3.07491179e-04
   3.90229048e-04  -1.39828111e-04

In [11]:
# the scores are unnormalized log probabilities. compute the probabilities
p = np.exp(y) / np.sum(np.exp(y))
print p.ravel()
print 'probabilities sum to ', p.sum()

[ 0.0149254   0.01492538  0.01492322  0.01492882  0.01492661  0.01491979
  0.01491983  0.01492979  0.01492958  0.01492467  0.01492687  0.01491679
  0.014918    0.0149293   0.01492734  0.01492749  0.01492836  0.01492378
  0.01492376  0.01492672  0.01492603  0.0149272   0.01492807  0.01492442
  0.01491435  0.01492446  0.01493275  0.01492774  0.01492335  0.01493085
  0.01492045  0.0149307   0.01491877  0.01491978  0.01492404  0.01492411
  0.01492682  0.01493193  0.01492581  0.01492773  0.01492154  0.01492943
  0.01492445  0.01492699  0.01492359  0.01493354  0.01492599  0.01492863
  0.01492753  0.01492877  0.0149275   0.01492046  0.01491974  0.01492607
  0.01492392  0.01492942  0.01493065  0.01492274  0.01492799  0.01491617
  0.01492572  0.01492165  0.01492793  0.01492059  0.01492138  0.0149305
  0.0149263 ]
probabilities sum to  1.0


In [12]:
print 'probability assigned to the correct next character is right now: ', p[ix_target,0]

probability assigned to the correct next character is right now:  0.0149216543899


In [13]:
loss = -np.log(p[ix_target,0])
print 'the cross-entropy (softmax) loss is ', loss

the cross-entropy (softmax) loss is  4.20494180632


In [14]:
# compute the gradient on y
dy = np.copy(p)
dy[ix_target] -= 1
print dy.ravel()
print 'sum of dy is ', dy.sum()
print 'the gradient for the correct character (%s) is: %s' % (ix_to_char[ix_target], dy[ix_target,0])
print 'the gradient for the character (a) is: ', dy[char_to_ix['a'],0]

[ 0.0149254   0.01492538  0.01492322  0.01492882  0.01492661  0.01491979
  0.01491983  0.01492979  0.01492958  0.01492467  0.01492687  0.01491679
  0.014918    0.0149293   0.01492734  0.01492749  0.01492836  0.01492378
  0.01492376  0.01492672  0.01492603  0.0149272   0.01492807  0.01492442
  0.01491435  0.01492446  0.01493275  0.01492774  0.01492335  0.01493085
  0.01492045  0.0149307   0.01491877  0.01491978  0.01492404  0.01492411
  0.01492682  0.01493193  0.01492581  0.01492773  0.01492154  0.01492943
  0.01492445  0.01492699  0.01492359  0.01493354  0.01492599  0.01492863
  0.01492753  0.01492877  0.0149275   0.01492046  0.01491974  0.01492607
  0.01492392  0.01492942  0.01493065  0.01492274  0.01492799  0.01491617
  0.01492572 -0.98507835  0.01492793  0.01492059  0.01492138  0.0149305
  0.0149263 ]
sum of dy is  1.45716771982e-16
the gradient for the correct character (t) is: -0.98507834561
the gradient for the character (a) is:  0.0149294265437


In [15]:
# we computed [y = np.dot(Why, h) + by]; Backpropagate to Why, h, and by
dWhy = np.dot(dy, h.T)
dh = np.dot(Why.T, dy)
dby = np.copy(dy)
print 'the hidden vector activations were:'
print h.ravel()
print 'the gradients are:'
print dh.ravel()
print 'the gradients dWhy have size: ', dWhy.shape
print 'a small sample is:'
print dWhy[:4,:4]

the hidden vector activations were:
[-0.00321813  0.00640499  0.00424507  0.00783602 -0.00428041  0.00900686
 -0.01618631  0.00071848 -0.01342806 -0.0048051 ]
the gradients are:
[-0.00983172 -0.01049054  0.00911112 -0.01110877 -0.00134586  0.00686193
 -0.01532285  0.0099507  -0.00530998  0.01267197]
the gradients dWhy have size:  (67, 10)
a small sample is:
[[ -4.80319012e-05   9.55970187e-05   6.33594305e-05   1.16955768e-04]
 [ -4.80318571e-05   9.55969309e-05   6.33593723e-05   1.16955661e-04]
 [ -4.80249076e-05   9.55830995e-05   6.33502052e-05   1.16938739e-04]
 [ -4.80429242e-05   9.56189576e-05   6.33739710e-05   1.16982609e-04]]


In [16]:
# we computed [h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h_prev + bh))]; 
# Backprop into Wxh, x, Whh, h_prev, bh:
dh_before_tanh = (1-h**2)*dh
dbh = np.copy(dh_before_tanh)
dWxh = np.dot(dh_before_tanh, x.T)
dWhh = np.dot(dh_before_tanh, h.T)
dh_prev = np.dot(Whh.T, dh_before_tanh)
print 'small sample of Whh:'
print Whh[:4,:4]

small sample of Whh:
[[-0.0028736  -0.00195895  0.00885911 -0.00349354]
 [-0.00773272 -0.00051873  0.00219991 -0.00234756]
 [ 0.01687054 -0.01221995  0.00125455 -0.00568523]
 [-0.00031645  0.00514377  0.01194564  0.0070584 ]]


In [17]:
# we now have the gradients for all parameters! (Wxh, Whh, Why, bh, by)
# lets do a parameter update
learning_rate = 0.1
Wxh2 = Wxh - learning_rate * dWxh
Whh2 = Whh - learning_rate * dWhh
Why2 = Why - learning_rate * dWhy
bh2 = bh - learning_rate * dbh
by2 = by - learning_rate * dby

In [18]:
# these parameters should be much better! lets try it out:
h2 = np.tanh(np.dot(Wxh2, x) + np.dot(Whh2, h_prev + bh2))
y2 = np.dot(Why2, h2) + by2
p2 = np.exp(y2) / np.sum(np.exp(y2))
print 'probability assigned to the correct next character was: ', p[ix_target,0]
print 'probability assigned to the correct next character is now: ', p2[ix_target,0]
loss2 = -np.log(p2[ix_target,0])
print 'the cross-entropy (softmax) loss was ', loss
print 'the loss is now ', loss2

probability assigned to the correct next character was:  0.0149216543899
probability assigned to the correct next character is now:  0.0164678301753
the cross-entropy (softmax) loss was  4.20494180632
the loss is now  4.10634648754


In [19]:
# note: the probability for the correct character went up! (and the loss went down)

In [20]:
# putting it together with loops
def lossFun(inputs, targets, hprev):
    """
    inputs,targets are both list of integers.
    hprev is Hx1 array of initial hidden state
    returns the loss, gradients on model parameters, and last hidden state
    """
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0
    
    # forward pass
    for t in xrange(len(inputs)):
        xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
        ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
        loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
    
    # backward pass: compute gradients going backwards
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    for t in reversed(xrange(len(inputs))):
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1 # backprop into y
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(Why.T, dy) + dhnext # backprop into h
        dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(Whh.T, dhraw)
        
    # clip to mitigate exploding gradients
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam)
    
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [21]:
loss, dWxh, dWhh, dWhy, dbh, dby, hnew = lossFun(inputs, targets, h_prev)
print loss

105.118896505


In [22]:
# TODO: write the sampling code
def sample(h, seed_ix, n):
    """ 
    sample a sequence of integers from the model 
    h is initial memory state, seed_ix is seed letter for first time step
    n is the number of time steps to sample for
    """
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = [] # sampled indices
    for t in xrange(n):
        pass # TODO: run the RNN for one time step, sample from distribution
    return ixes


In [23]:
# Stochastic Gradient Descent
n, p = 0, 0
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
learning_rate = 1e-3
while n < 10000:
    # prepare inputs (we're sweeping from left to right in steps seq_length long)
    if p+seq_length+1 >= len(data) or n == 0: 
        hprev = np.zeros((hidden_size,1)) # reset RNN memory
        p = 0 # go from start of data
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

    # forward seq_length characters through the net and fetch gradient
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    if n % 100 == 0: print 'iter %d, loss: %f' % (n, smooth_loss) # print progress

    # perform parameter update with Adagrad
    for param, dparam in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby]):
        param += -learning_rate * dparam

    p += seq_length # move data pointer
    n += 1 # iteration counter 

iter 0, loss: 105.117314
iter 100, loss: 104.983384
iter 200, loss: 104.623511
iter 300, loss: 104.093896
iter 400, loss: 103.407954
iter 500, loss: 102.742497
iter 600, loss: 101.886946
iter 700, loss: 101.017334
iter 800, loss: 100.224049
iter 900, loss: 99.275074
iter 1000, loss: 98.310447
iter 1100, loss: 97.280136
iter 1200, loss: 96.170347
iter 1300, loss: 95.026980
iter 1400, loss: 94.074431
iter 1500, loss: 93.214796
iter 1600, loss: 92.194884
iter 1700, loss: 91.461468
iter 1800, loss: 90.884004
iter 1900, loss: 90.051538
iter 2000, loss: 89.025933
iter 2100, loss: 88.348015
iter 2200, loss: 87.853860
iter 2300, loss: 87.232155
iter 2400, loss: 86.868957
iter 2500, loss: 86.633172
iter 2600, loss: 86.099010
iter 2700, loss: 85.847026
iter 2800, loss: 85.920993
iter 2900, loss: 85.654873
iter 3000, loss: 85.194774
iter 3100, loss: 85.314185
iter 3200, loss: 85.286007
iter 3300, loss: 85.060589
iter 3400, loss: 84.918822
iter 3500, loss: 84.606756
iter 3600, loss: 84.445501
iter

In [24]:
# TODO: Implement a sampling function that lets us generate from the model.